# Assignment4 RAG (ollama-gemma2)

In [1]:
from langchain.chat_models.ollama import ChatOllama
from langchain.chat_models.openai import ChatOpenAI
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

# LLM_model, models = ["openai", "gpt-4o-mini-2024-07-18"]
LLM_model, models = ["ollama", "llama3.1:latest"]

file_name = "document.txt"

llm = (
    ChatOllama(temperature=0.1, model=models)
    if LLM_model == "ollama"
    else ChatOpenAI(temperature=0.1, model=models)
)

loader = UnstructuredFileLoader(f"../files/{file_name}")
cache_dir = LocalFileStore(f"../.cache/embeddings/{LLM_model}/{models}/{file_name}")

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    separators=["\n\n", ".", "?", "!"],
    chunk_size=1000,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)
embeddings = (
    OllamaEmbeddings(model=models) if LLM_model == "ollama" else OpenAIEmbeddings()
)

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
    memory_key="history",
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            당신은 문서를 읽고 대답해주는 AI입니다. 주어지는 문서를 통해 답변을 주세요. 
            만약 정보가 문서에 없다면, '모르겠습니다'라고 답변해주세요.
            한국어로 답변해주세요.
            
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": RunnableLambda(load_memory),
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question).content
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print(result)

In [2]:
invoke_chain("Is Aaronson guilty?")

The provided text excerpts from George Orwell's *Nineteen Eighty-Four* heavily imply that the Party, and O'Brien in particular, are manipulating reality and using torture to extract confessions and enforce their ideology. 

**Based on the text, we cannot definitively say whether Aaronson is guilty or not.** The Party's methods are designed to break individuals and force them to confess to whatever the Party deems necessary. 

Here's why we can't trust the information presented:

* **O'Brien's words are manipulative:** He states that "the object of persecution is persecution" and "the object of torture is torture." This suggests that the Party's goal is not to uncover truth but to instill fear and obedience.
* **Reality is controlled:** O'Brien claims that "Reality is inside the skull" and that the Party "makes the laws of Nature." This implies that the Party can control what people perceive as truth.
* **Winston's state:** Winston is being tortured and broken down. He is unable to thin

In [3]:
invoke_chain("What message did he write in the table?")

The text you provided doesn't mention Winston writing anything in a table. 


Let me know if you have any other questions about *Nineteen Eighty-Four* or need further clarification on any part of the text! 



In [4]:
invoke_chain("Who is Julia?")

Julia is a young woman who becomes Winston's lover in *Nineteen Eighty-Four*. She is described as rebellious, independent, and passionate. Unlike Winston, she seems more focused on enjoying the present moment and indulging in physical pleasures as a way to resist the Party's control. 

Here are some key things to know about Julia:

* **She is a member of the Outer Party:** This means she works a regular job and lives under the Party's constant surveillance, but she manages to find ways to rebel against the system.
* **She is resourceful and cunning:** She has a network of contacts and knows how to obtain forbidden items, like chocolate and perfume.
* **She is sexually liberated:** She enjoys physical intimacy and sees it as a way to assert her independence and challenge the Party's control over sexuality.
* **She is initially more pragmatic than Winston:** While Winston is driven by a desire to understand the truth and fight against the Party, Julia is more focused on living a fulfilli

In [5]:
invoke_chain("What was the first question I asked?")

invoke_chain("What was the second question I asked?")

invoke_chain("What was the third question I asked?")

The first question you asked was:  "Is Aaronson guilty?" 

The second question you asked was: "What message did he write in the table?" 


Let me know if you have any other questions! 

The third question you asked was: "Who is Julia?" 



In [6]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='The provided text excerpts from George Orwell\'s *Nineteen Eighty-Four* heavily imply that the Party, and O\'Brien in particular, are manipulating reality and using torture to extract confessions and enforce their ideology. \n\n**Based on the text, we cannot definitively say whether Aaronson is guilty or not.** The Party\'s methods are designed to break individuals and force them to confess to whatever the Party deems necessary. \n\nHere\'s why we can\'t trust the information presented:\n\n* **O\'Brien\'s words are manipulative:** He states that "the object of persecution is persecution" and "the object of torture is torture." This suggests that the Party\'s goal is not to uncover truth but to instill fear and obedience.\n* **Reality is controlled:** O\'Brien claims that "Reality is inside the skull" and that the Party "makes the laws of Nature." This implie